<a href="https://colab.research.google.com/github/EthanMerrill/Trading-Algos/blob/master/RiskOnRiskOffStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install alpaca_trade_api
!pip install alpaca-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 13.6 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.5
    Uninstalling msgpack-1.0.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 36.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.1:
      Successfully uninstalled pydantic-2.2.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.0 which is incompatible.


In [4]:
import pandas_datareader.data as web
import pandas as pd
import alpaca_trade_api as alpaca
import requests
import base64

In [5]:
APCA_API_BASE_URL="https://paper-api.alpaca.markets"
ALPCA_PAPER_KEY = "PKIDK8BXVS07HWAKUYCS"
ALPCA_PAPER_SECRET = "zmSekWcDtkbnMVeadoHmuqCgw5edeNLgKxjIIcW6"

In [6]:
# Importing the API and instantiating the REST client according to our keys
from alpaca.trading.client import TradingClient




10000
10000


In [53]:
from pandas_datareader.av import AlphaVantage
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.data import StockHistoricalDataClient
client = StockHistoricalDataClient(ALPCA_PAPER_KEY, ALPCA_PAPER_SECRET)


trading_client = TradingClient(AlphaVantage, ALPCA_PAPER_SECRET, paper=True)

def Orderer (symbol, quantity, buy):
  print(f'order recieved to {buy if "buy" else "sell"} {quantity} of {symbol}')
  # preparing orders
  market_order_data = MarketOrderRequest(
                    symbol=symbol,
                    qty=quantity,
                    side= OrderSide.BUY if buy else OrderSide.SELL,
                    time_in_force=TimeInForce.DAY
                    )
  # Market order
  market_order = trading_client.submit_order(
                order_data=market_order_data
               )


## determines how many shares of each item in the portfolio need to be purchased given their current prices
def ShareAllocator(totalCash, portfolio_object):
  for ticker in portfolio_object:
        print(ticker, portfolio_object[ticker])
        quote = client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=[ticker]))
        quote_price = quote[ticker].ask_price
        share_count = totalCash*portfolio_object[ticker]/quote_price
        portfolio_object[ticker] = {"%":portfolio_object[ticker], "shares":round(share_count)}

  return portfolio_object

def SellEverything(portfolio):
      # Print the quantity of shares for each position.
    for position in portfolio:

      print("Selling {} shares of {}".format(position.qty, position.symbol))
      Orderer(position.symbol, position.qty, False)

def AssetAllocator(desired_portfolio, current_positions, cash):
  #if ANY of the desired positions are already held in the account, abort
  print(current_positions, desired_portfolio)
  for ticker in current_positions:
    if ticker in desired_portfolio:
      print(f'ticker {ticker} already owned! Ending')
      return
    ## Sell Everything in portfolio currently
  SellEverything(portfolio)
  # determine how many shares to purchase
  desired_portfolio = ShareAllocator(int(cash), desired_portfolio)
  for ticker in desired_portfolio:
    Orderer(ticker, conserv_portfolio[ticker]["shares"], True)

In [54]:
from sqlalchemy import null
# Get historical returns for selected assets
BILDF = web.DataReader('BIL', 'stooq')
BNDDF = web.DataReader('BND', 'stooq')
QQQDF = web.DataReader('QQQ', 'stooq')
UPRODF = web.DataReader('UPRO', 'stooq')
EIFDF = web.DataReader('EIF', 'stooq')

# Calculate returns in the past 60 days for BIL

cumBIL = BILDF["Open"].iloc[0]-BILDF["Open"].iloc[60];
cumBND = BNDDF["Open"].iloc[0]-BNDDF["Open"].iloc[60];


# create the trading client
trading_client = TradingClient(ALPCA_PAPER_KEY, ALPCA_PAPER_SECRET, paper=True)
# Get a list of all of our positions.
portfolio = trading_client.get_all_positions()
account = trading_client.get_account()
print(account.cash)

if cumBND>cumBIL:
    print('Buy Signal')
    ## Sell Everything in portfolio currently
    SellEverything(portfolio)
    ## Buy the Aggressive stocks
    agro_portfolio = {
        "QQQ":45,
        "UPRO":45
    }
    AssetAllocator(agro_portfolio, trading_client.get_all_positions(), account.cash)

else:
    print("Sell Signal")

    ## Buy the conservative stock
    conserv_portfolio = {
        "IEF":.95
    }
    AssetAllocator(conserv_portfolio, trading_client.get_all_positions(), account.cash)


10000
Sell Signal
[] {'IEF': 0.95}
IEF 0.95
{'IEF': {'%': 0.95, 'shares': 102}}
order recieved to True 102 of IEF


In [34]:
trading_client.get_all_positions()

[]

QQQ 0.45
buy 12 shares of QQQ at 383.45
UPRO 0.45
buy 90 shares of UPRO at 50.25
{'QQQ': {'%': 0.45, 'Shares': 12}, 'UPRO': {'%': 0.45, 'Shares': 90}}


In [27]:
    api = alpaca.REST("PKFOXX7NR9BPZ70VS5QD", "hRVqGr0lAdSsKN5u1s8k45x5qJAYdZg49irheS56", APCA_API_BASE_URL)


In [13]:
api.get_account().last_equity()

NameError: ignored